# DragonNet

__Reference__: [Claudia Shi et al, Adapting Neural Networks for the Estimation of Treatment Effects, NeurIPS 2019](https://arxiv.org/pdf/1906.02120v2.pdf)

__Implementation remarks__: our implementation is exactly the same of the original paper with the exception 
    of a _sklearn.preprocessing.StandardScaler_ which was originally used to scale predictions. 

## DragonNet on IHDP 

In [1]:
from causalforge.model import Model , PROBLEM_TYPE
from causalforge.data_loader import DataLoader 

# load IHDP dataset 
r = DataLoader.get_loader('IHDP').load()
X_tr, T_tr, YF_tr, YCF_tr, mu_0_tr, mu_1_tr, X_te, T_te, YF_te, YCF_te, mu_0_te, mu_1_te = r

# model 
params={}
params['input_dim'] = X_tr.shape[1] 
    
    
dragonnet = Model.create_model("dragonnet",
                               params,
                               problem_type=PROBLEM_TYPE.CAUSAL_TREATMENT_EFFECT_ESTIMATION, 
                               multiple_treatments=False)

dragonnet.model.summary()

2023-05-05 16:08:33.088174: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 25)]         0           []                               
                                                                                                  
 dense (Dense)                  (None, 200)          5200        ['input[0][0]']                  
                                                                                                  
 dense_1 (Dense)                (None, 200)          40200       ['dense[0][0]']                  
                                                                                                  
 dense_2 (Dense)                (None, 200)          40200       ['dense_1[0][0]']                
                                                                                              

2023-05-05 16:08:36.229178: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


 dense_4 (Dense)                (None, 100)          20100       ['dense_2[0][0]']                
                                                                                                  
 dense_5 (Dense)                (None, 100)          20100       ['dense_2[0][0]']                
                                                                                                  
 dense_6 (Dense)                (None, 100)          10100       ['dense_4[0][0]']                
                                                                                                  
 dense_7 (Dense)                (None, 100)          10100       ['dense_5[0][0]']                
                                                                                                  
 dense_3 (Dense)                (None, 1)            201         ['dense_2[0][0]']                
                                                                                                  
 y0_predic

In [2]:
from causalforge.metrics import eps_ATE_diff, PEHE_with_ite
import numpy as np

experiment_ids = [1,10,400]

eps_ATE_tr, eps_ATE_te = [], []
eps_PEHE_tr, eps_PEHE_te = [] , [] 



for idx in experiment_ids:    
    t_tr, y_tr, x_tr, mu0tr, mu1tr = T_tr[:,idx] , YF_tr[:,idx], X_tr[:,:,idx], mu_0_tr[:,idx], mu_1_tr[:,idx] 
    t_te, y_te, x_te, mu0te, mu1te = T_te[:,idx] , YF_te[:,idx], X_te[:,:,idx], mu_0_te[:,idx], mu_1_te[:,idx]  
    
    
    # Train your causal method on train-set ...
    dragonnet.fit(x_tr,t_tr,y_tr)

    # Validate your method test-set ... 
    ATE_truth_tr = (mu1tr - mu0tr).mean()
    ATE_truth_te = (mu1te - mu0te).mean()
    
    ITE_truth_tr = (mu1tr - mu0tr)
    ITE_truth_te = (mu1te - mu0te)
    
    eps_ATE_tr.append( eps_ATE_diff( dragonnet.predict_ite(x_tr), ITE_truth_tr) )
    eps_ATE_te.append( eps_ATE_diff( dragonnet.predict_ite(x_te), ITE_truth_te) )
    
    eps_PEHE_tr.append( PEHE_with_ite( dragonnet.predict_ite(x_tr), ITE_truth_tr, sqrt=True))
    eps_PEHE_te.append( PEHE_with_ite(dragonnet.predict_ite(x_te), ITE_truth_te , sqrt=True))
        

Epoch 1/30


/Users/AG62216/opt/anaconda3/envs/causalforge/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


9/9 [==============================] - 2s 43ms/step - loss: 1837.5880 - regression_loss: 847.2359 - binary_classification_loss: 35.7781 - treatment_accuracy: 0.7672 - track_epsilon: 0.0607 - val_loss: 402.8440 - val_regression_loss: 140.5852 - val_binary_classification_loss: 21.5544 - val_treatment_accuracy: 0.8006 - val_track_epsilon: 0.0610 - lr: 0.0010
Epoch 2/30
9/9 [==============================] - 0s 7ms/step - loss: 355.1468 - regression_loss: 146.8956 - binary_classification_loss: 30.7850 - treatment_accuracy: 0.8042 - track_epsilon: 0.0616 - val_loss: 186.1139 - val_regression_loss: 57.0438 - val_binary_classification_loss: 20.9781 - val_treatment_accuracy: 0.8006 - val_track_epsilon: 0.0606 - lr: 0.0010
Epoch 3/30
9/9 [==============================] - 0s 6ms/step - loss: 254.1957 - regression_loss: 107.0528 - binary_classification_loss: 30.1929 - treatment_accuracy: 0.8069 - track_epsilon: 0.0597 - val_loss: 217.4558 - val_regression_loss: 70.0112 - val_binary_classificatio

/Users/AG62216/opt/anaconda3/envs/causalforge/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/gradient_descent.py:111: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


9/9 [==============================] - 2s 46ms/step - loss: 211.6185 - regression_loss: 84.1101 - binary_classification_loss: 29.5777 - treatment_accuracy: 0.8096 - track_epsilon: 0.0494 - val_loss: 187.8485 - val_regression_loss: 57.8545 - val_binary_classification_loss: 20.9967 - val_treatment_accuracy: 0.8006 - val_track_epsilon: 0.0354 - lr: 1.0000e-05
Epoch 2/100
9/9 [==============================] - 0s 8ms/step - loss: 201.3698 - regression_loss: 79.9347 - binary_classification_loss: 29.4149 - treatment_accuracy: 0.8123 - track_epsilon: 0.0361 - val_loss: 173.2833 - val_regression_loss: 52.0208 - val_binary_classification_loss: 20.9611 - val_treatment_accuracy: 0.8006 - val_track_epsilon: 0.0361 - lr: 1.0000e-05
Epoch 3/100
9/9 [==============================] - 0s 7ms/step - loss: 195.1406 - regression_loss: 76.8607 - binary_classification_loss: 29.3787 - treatment_accuracy: 0.8096 - track_epsilon: 0.0349 - val_loss: 179.8145 - val_regression_loss: 54.6517 - val_binary_classifi

...

9/9 [==============================] - 0s 6ms/step - loss: 136.5742 - regression_loss: 49.5240 - binary_classification_loss: 27.7873 - treatment_accuracy: 0.7970 - track_epsilon: 0.0029 - val_loss: 159.8507 - val_regression_loss: 48.9664 - val_binary_classification_loss: 17.2532 - val_treatment_accuracy: 0.9062 - val_track_epsilon: 0.0029 - lr: 7.8125e-08
Epoch 65/100
9/9 [==============================] - 0s 7ms/step - loss: 136.9658 - regression_loss: 49.5204 - binary_classification_loss: 27.7872 - treatment_accuracy: 0.8078 - track_epsilon: 0.0029 - val_loss: 160.0629 - val_regression_loss: 49.0566 - val_binary_classification_loss: 17.2534 - val_treatment_accuracy: 0.9062 - val_track_epsilon: 0.0029 - lr: 7.8125e-08
Epoch 66/100
9/9 [==============================] - 0s 6ms/step - loss: 137.6142 - regression_loss: 49.5337 - binary_classification_loss: 27.7871 - treatment_accuracy: 0.7997 - track_epsilon: 0.0029 - val_loss: 160.1727 - val_regression_loss: 49.1039 - val_binary_classif

Epoch 11/30
9/9 [==============================] - 0s 6ms/step - loss: 242.7726 - regression_loss: 98.6682 - binary_classification_loss: 28.0497 - treatment_accuracy: 0.7965 - track_epsilon: 0.0117 - val_loss: 298.7372 - val_regression_loss: 199.0750 - val_binary_classification_loss: 18.0941 - val_treatment_accuracy: 0.8586 - val_track_epsilon: 0.0117 - lr: 0.0010
Epoch 12/30
9/9 [==============================] - 0s 6ms/step - loss: 215.9338 - regression_loss: 87.1851 - binary_classification_loss: 28.0315 - treatment_accuracy: 0.7910 - track_epsilon: 0.0116 - val_loss: 283.0320 - val_regression_loss: 206.1786 - val_binary_classification_loss: 18.0007 - val_treatment_accuracy: 0.8586 - val_track_epsilon: 0.0116 - lr: 0.0010
Epoch 13/30
9/9 [==============================] - 0s 6ms/step - loss: 219.0686 - regression_loss: 88.5876 - binary_classification_loss: 27.9406 - treatment_accuracy: 0.8009 - track_epsilon: 0.0115 - val_loss: 268.7195 - val_regression_loss: 178.6040 - val_binary_cl

9/9 [==============================] - 0s 5ms/step - loss: 181.1911 - regression_loss: 72.5168 - binary_classification_loss: 27.4537 - treatment_accuracy: 0.7938 - track_epsilon: 0.0130 - val_loss: 235.4143 - val_regression_loss: 132.5860 - val_binary_classification_loss: 18.0358 - val_treatment_accuracy: 0.9062 - val_track_epsilon: 0.0165 - lr: 1.0000e-05
Epoch 13/100
9/9 [==============================] - 0s 6ms/step - loss: 179.1568 - regression_loss: 70.2396 - binary_classification_loss: 27.4581 - treatment_accuracy: 0.8009 - track_epsilon: 0.0153 - val_loss: 251.7753 - val_regression_loss: 150.2389 - val_binary_classification_loss: 17.9469 - val_treatment_accuracy: 0.9115 - val_track_epsilon: 0.0153 - lr: 1.0000e-05
Epoch 14/100
9/9 [==============================] - 0s 6ms/step - loss: 195.4511 - regression_loss: 78.3698 - binary_classification_loss: 27.4244 - treatment_accuracy: 0.7928 - track_epsilon: 0.0158 - val_loss: 226.2286 - val_regression_loss: 114.7993 - val_binary_clas

1/9 [==>...........................] - ETA: 0s - loss: 167.4697 - regression_loss: 67.0101 - binary_classification_loss: 28.9756 - treatment_accuracy: 0.8281 - track_epsilon: 0.0149
Epoch 34: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-06.
9/9 [==============================] - 0s 7ms/step - loss: 172.7515 - regression_loss: 66.9702 - binary_classification_loss: 27.2768 - treatment_accuracy: 0.7982 - track_epsilon: 0.0150 - val_loss: 219.5257 - val_regression_loss: 113.2501 - val_binary_classification_loss: 17.9196 - val_treatment_accuracy: 0.9115 - val_track_epsilon: 0.0155 - lr: 1.0000e-05
Epoch 35/100
9/9 [==============================] - 0s 7ms/step - loss: 157.8652 - regression_loss: 59.2994 - binary_classification_loss: 27.2410 - treatment_accuracy: 0.7945 - track_epsilon: 0.0159 - val_loss: 217.7503 - val_regression_loss: 108.6164 - val_binary_classification_loss: 18.0117 - val_treatment_accuracy: 0.9115 - val_track_epsilon: 0.0151 - lr: 5.0000e-06
Epoch 36/1

Epoch 54/100
9/9 [==============================] - 0s 6ms/step - loss: 150.2748 - regression_loss: 55.8531 - binary_classification_loss: 27.1397 - treatment_accuracy: 0.7972 - track_epsilon: 0.0146 - val_loss: 218.1107 - val_regression_loss: 106.8210 - val_binary_classification_loss: 18.0531 - val_treatment_accuracy: 0.9062 - val_track_epsilon: 0.0147 - lr: 1.2500e-06
Epoch 55/100
9/9 [==============================] - 0s 6ms/step - loss: 151.2827 - regression_loss: 55.8578 - binary_classification_loss: 27.1395 - treatment_accuracy: 0.7891 - track_epsilon: 0.0144 - val_loss: 216.6774 - val_regression_loss: 106.8535 - val_binary_classification_loss: 18.0444 - val_treatment_accuracy: 0.9062 - val_track_epsilon: 0.0143 - lr: 1.2500e-06
Epoch 56/100
9/9 [==============================] - 0s 6ms/step - loss: 146.7676 - regression_loss: 55.8992 - binary_classification_loss: 27.1380 - treatment_accuracy: 0.7874 - track_epsilon: 0.0144 - val_loss: 223.9153 - val_regression_loss: 107.0756 - va

## Results 

In [3]:
import pandas as pd 

pd.DataFrame([[np.mean(eps_ATE_tr),np.mean(eps_ATE_te),np.mean(eps_PEHE_tr),np.mean(eps_PEHE_te)]],
             columns=['eps_ATE_tr','eps_ATE_te','eps_PEHE_tr','eps_PEHE_te'], 
             index=['DragonNet'])

,eps_ATE_tr,eps_ATE_te,eps_PEHE_tr,eps_PEHE_te
DragonNet,0.091358,0.080693,0.654876,0.649264
